In [1]:
#| default_exp analysis

# EVCI analysis

> **API**: The utilization scores for each site are computed here.

In [2]:
#|hide
from nbdev.showdoc import *

## Import libraries

In [3]:
#|export

import numpy as np
import pandas as pd
import geopandas as gpd

import shapely

import os
from tqdm import tqdm

import matplotlib.pyplot as plt

from scipy.cluster.vq import kmeans2, whiten
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster

from evci_tool.config import *
from evci_tool.model import *

import warnings
warnings.filterwarnings("ignore")

In [10]:
#|export

def run_episode(m,s,t,g,s_df,txt,OUTPUT_PATH,corridor):
    "This function runs a full episode of analysis on a set of sites."
    
    print('\n' + txt.capitalize() + ' Analysis')
    print('________________\n')
    total = s_df.shape[0]
    
    #s_df = s_df[s_df['year 1'] == 1]
    #s_df = s_df.reset_index(drop=True)
    
    Nc = s_df.shape[0]
    print(f'Number of sites: {Nc}/{total}')

    #@title Compute scores

    backoff_factor = 2 #@param {type:"slider", min:1, max:5, step:1}

    u_df = run_analysis(m,s,t,g,s_df,backoff_factor=backoff_factor)

    print(f'Total capex charges = INR Cr {sum(u_df.capex)/1e7:.2f}')
    print(f'Total opex charges = INR Cr {sum(u_df.opex)/1e7:.2f}')
    print(f'Total Margin = INR Cr {sum(u_df.margin)/1e7:.2f}')        

    #@title Prepare data
    s_u_df = s_df.copy()

    s_u_df['utilization'] = u_df.utilization
    s_u_df['unserviced'] = u_df.unserviced
    s_u_df['capex'] = u_df.capex
    s_u_df['opex'] = u_df.opex
    s_u_df['margin'] = u_df.margin
    s_u_df['max vehicles'] = u_df['max vehicles']
    s_u_df['estimated vehicles'] = u_df['estimated vehicles']

    #@title Save initial analysis to Excel
    output_df = s_u_df.copy()
    output_df.drop('geometry', axis=1, inplace=True)
    output_df.to_excel(OUTPUT_PATH + '/' + txt + '_evci_analysis.xlsx')
    return s_u_df

`Arguments`:

1. `m`: dataframe of model parameters (from model.xlsx)
2. `s`: dataframe of sites (from sites.xlsx)
3. `t`: dataframe of traffic profile (from traffic.xlsx)
4. `g`: dataframe of grid parameters (from grid.xlsx)
5. `s_df`: pre-processed geopandas dataframe with each point stored as shapely point object
6. `txt`: a string that identifies the episode (e.g. initial, final, with_cluster etc)
7. `OUTPUT_PATH`: the directory path where the generated output files will be stored
8. `corridor`: a string that identifies the corridor being analyzed (e.g. chandigarh_leh)

`Returns`: 

A pandas dataframe: `s_u_df` is a dataframe with computed utilization values for each site.

In [5]:
#|export

def analyze_sites(corridor:str, cluster:bool=False):
    "The function analyzes sites specified as part of a corridor."

    #@title Read data from excel sheets
    model,site,traffic,grid, INPUT_PATH, OUTPUT_PATH = setup_and_read_data(corridor)

    #@title Read required data sheets only
    df = gpd.read_file(INPUT_PATH + '/shape_files/' + corridor + '.shp')

    data = site['sites']
    data['Name'] = data['Name']
    data['Latitude'] = pd.to_numeric(data['Latitude'])
    data['Longitude'] = pd.to_numeric(data['Longitude'])
    data['geometry'] = [shapely.geometry.Point(xy) for xy in 
                        zip(data['Longitude'], data['Latitude'])]

    data_df = {}

    data_df = gpd.GeoDataFrame(data, geometry=data['geometry'])

    s_df = pd.DataFrame(columns=['Name',
                                'Latitude', 'Longitude',
                                'Traffic congestion',
                                'year 1',
                                'kiosk hoarding',
                                'hoarding margin',
                                'geometry'])

    s_df = s_df.reset_index(drop=True)

    for i in range(data_df.shape[0]):
        s_df.loc[i] = [
           data_df.loc[i].Name, 
           data_df.loc[i].Latitude, 
           data_df.loc[i].Longitude, 
           data_df.loc[i]['Traffic congestion'],
           data_df.loc[i]['Year for Site recommendation'],
           data_df.loc[i]['Hoarding/Kiosk (1 is yes & 0 is no)'],
           data_df.loc[i]['Hoarding margin'],
           data_df.loc[i].geometry
        ] 
    
    s_u_df = run_episode(model,site,traffic,grid,s_df,'initial',OUTPUT_PATH, corridor)

    #@title Threshold and cluster
    if cluster:
        clustering_candidates = s_u_df[(s_u_df.utilization <= 0.2) & (s_u_df['year 1'] == 1)]
        points = np.array((clustering_candidates.apply(lambda x: list([x['Latitude'], x['Longitude']]),axis=1)).tolist())
        Z = linkage (points, method='complete', metric='euclidean');
        plt.figure(figsize=(14,8))
        dendrogram(Z);
        max_d = 0.01
        clusters = fcluster(Z, t=max_d, criterion='distance')
        clustered_candidates = gpd.GeoDataFrame(clustering_candidates)
        #base = grid_df.plot(color='none', alpha=0.2, edgecolor='black', figsize=(8,8))
        #clustered_candidates.plot(ax=base, column=clusters, legend=True)

    #@title Build final list of sites
    confirmed_sites = s_u_df[s_u_df.utilization > 0.2]
    if cluster:
        val, ind = np.unique (clusters, return_index=True)
        clustered_sites = clustered_candidates.reset_index(drop=True)
        clustered_sites = clustered_sites.iloc[clustered_sites.index.isin(ind)]
        final_list_of_sites = pd.concat([confirmed_sites, clustered_sites], axis=0)
    else:
        final_list_of_sites = confirmed_sites.copy()

    if cluster:
        s_u_df = run_episode(final_list_of_sites,'cluster',OUTPUT_PATH)

`Arguments`:

1. `corridor`: a string that identifies the corridor being analyzed (e.g. chandigarh_leh)
2. `cluster`: a boolean flag that indicates whether clustering algorithm should be run. Default is `False`

`Returns`:

None.

In [6]:
analyze_sites('chandigarh_leh')


Initial Analysis
________________

Number of sites: 48/48


100%|███████████████████████████████████████████| 48/48 [00:00<00:00, 748425.99it/s]

Total capex charges = INR Cr 0.19
Total opex charges = INR Cr 1.01
Total Margin = INR Cr nan
